In [1]:
import pandas as pd
import plotly.express as px
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from datetime import timedelta
import plotly.graph_objects as go

### Cargando Data

In [2]:
df_sales = pd.read_csv('data/daily_sales.csv')
df_factors = pd.read_csv('data/external_factors.csv')
df_hierarchy = pd.read_csv('data/product_hierarchy.csv')
df_promotion = pd.read_csv('data/promotional_calendar.csv')
df_inventary = pd.read_csv('data/store_inventory.csv')

df_sales['fecha'] = pd.to_datetime(df_sales['fecha'], format='%Y-%m-%d')
df_factors['fecha'] = pd.to_datetime(df_factors['fecha'], format='%Y-%m-%d')
df_promotion['fecha_inicio'] = pd.to_datetime(df_promotion['fecha_inicio'], format='%Y-%m-%d')
df_promotion['fecha_fin'] = pd.to_datetime(df_promotion['fecha_fin'], format='%Y-%m-%d')
df_inventary['fecha_inicio'] = pd.to_datetime(df_inventary['fecha_inicio'], format='%Y-%m-%d')


### Creando Features

In [4]:
df_sales_factor = pd.merge(df_sales, df_factors, on=['region', 'fecha'], how='inner')

# Agregar columnas derivadas de la fecha
df_sales_factor['año'] = df_sales_factor['fecha'].dt.year
df_sales_factor['mes'] = df_sales_factor['fecha'].dt.month
df_sales_factor['dia'] = df_sales_factor['fecha'].dt.day
df_sales_factor['dia_semana'] = df_sales_factor['fecha'].dt.weekday  # 0 = lunes
df_sales_factor['semana'] = df_sales_factor['fecha'].dt.isocalendar().week
# df_sales_factor['dia_agno_pasado'] = df_sales_factor['fecha'] - timedelta(days=364)

### Haciendo modelos

In [5]:
resultados = {}
df_predicciones = pd.DataFrame()
products_list=list(df_sales['sku'].unique())
modelos = {}

for sku in products_list:
    print(f"Entrenando modelo para {sku}...")
    
    df_sku = df_sales_factor[df_sales_factor['sku'] == sku].copy()
    fechas = df_sku['fecha']
    unidades = df_sku['unidades']

    # Variables predictoras (features)
    features = ['region', 'tienda', 'familia', 'precio_unitario',
                'temperatura', 'indice_economico', 'año', 'mes', 'dia', 'dia_semana', 'semana']
    
    # One-hot encoding para variables categóricas
    df_model = pd.get_dummies(df_sku[features + ['unidades']], drop_first=True)

    X = df_model.drop('unidades', axis=1)
    y = df_model['unidades']

    # Split train/test
    original_indices = df_sku.index
    X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, original_indices, test_size=0.2, random_state=42
)
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Modelo
    model = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    # Evaluación
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    resultados[sku] = rmse

    print(f"RMSE para {sku}: {rmse:.2f}")
    df_preds = pd.DataFrame({
        'fecha': df_sku.loc[idx_test, 'fecha'].values,
        'real': y_test.values,
        'pred': y_pred,
        'SKU': sku
    })
    modelos[sku] = model
    df_predicciones = pd.concat([df_predicciones, df_preds], ignore_index=True)

Entrenando modelo para SKU_01...
RMSE para SKU_01: 121.79
Entrenando modelo para SKU_02...
RMSE para SKU_02: 94.54
Entrenando modelo para SKU_03...
RMSE para SKU_03: 61.01
Entrenando modelo para SKU_04...
RMSE para SKU_04: 63.96
Entrenando modelo para SKU_05...
RMSE para SKU_05: 59.16
Entrenando modelo para SKU_06...
RMSE para SKU_06: 28.93
Entrenando modelo para SKU_07...
RMSE para SKU_07: 53.58
Entrenando modelo para SKU_08...
RMSE para SKU_08: 38.77
Entrenando modelo para SKU_09...
RMSE para SKU_09: 30.97
Entrenando modelo para SKU_10...
RMSE para SKU_10: 34.94
Entrenando modelo para SKU_11...
RMSE para SKU_11: 43.38
Entrenando modelo para SKU_12...
RMSE para SKU_12: 25.50


### Resultado grafico por producto

In [22]:
sku_a_graficar = 'SKU_01'
df_plot = df_predicciones[df_predicciones['SKU'] == sku_a_graficar].sort_values('fecha')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_plot['fecha'], y=df_plot['real'],
    mode='lines+markers', name='Real'
))

fig.add_trace(go.Scatter(
    x=df_plot['fecha'], y=df_plot['pred'],
    mode='lines+markers', name='Predicho'
))

fig.update_layout(
    title=f'Predicción vs Real para {sku_a_graficar}',
    xaxis_title='Fecha',
    yaxis_title='Unidades',
    height=500
)

fig.show()


### Creado data para hacer predicciones a 7 días

In [ ]:
fechas_pred = pd.date_range(start='2025-02-01', end='2025-02-07')

tiendas = list(df_sales_factor['tienda'].unique())

# Crear todas las combinaciones posibles: tienda x producto x fecha
combinaciones = [(fecha, tienda, sku) for fecha in fechas_pred for tienda in tiendas for sku in products_list]

df_pred_base = pd.DataFrame(combinaciones, columns=['fecha', 'tienda', 'sku'])

df_tiendas_regiones = df_sales[['tienda', 'region']].drop_duplicates()

df_pred_base = df_pred_base.merge(df_tiendas_regiones, on='tienda', how='left')

In [8]:
df_ultimos_precios = df_sales[df_sales['fecha'] == '2024-12-30'][['tienda', 'sku', 'precio_unitario', 'familia']]
df_ultimos_precios = df_ultimos_precios.drop_duplicates(subset=['tienda', 'sku'])
df_pred_base = df_pred_base.merge(df_ultimos_precios, on=['tienda', 'sku'], how='left')


In [ ]:
# Supuestos de temperatura por región
temperaturas_region_dia = {
    '2025-02-01': {'Norte': 28.2, 'Centro': 29.3, 'Sur': 18.5},
    '2025-02-02': {'Norte': 32.0, 'Centro': 28.0, 'Sur': 19.0},
    '2025-02-03': {'Norte': 34.8, 'Centro': 30.7, 'Sur': 20.7},
    '2025-02-04': {'Norte': 30.5, 'Centro': 26.5, 'Sur': 21.3},
    '2025-02-05': {'Norte': 30.0, 'Centro': 27.0, 'Sur': 16.8},
    '2025-02-06': {'Norte': 30.3, 'Centro': 26.2, 'Sur': 17.1},
    '2025-02-07': {'Norte': 29.7, 'Centro': 25.8, 'Sur': 18.9}
}
df_pred_base['temperatura'] = df_pred_base.apply(
    lambda row: temperaturas_region_dia[row['fecha'].strftime('%Y-%m-%d')][row['region']],
    axis=1
)
# df_pred_base

In [10]:
df_pred_base['indice_economico']=96.1
df_pred_base['año'] = df_pred_base['fecha'].dt.year
df_pred_base['mes'] = df_pred_base['fecha'].dt.month
df_pred_base['dia'] = df_pred_base['fecha'].dt.day
df_pred_base['dia_semana'] = df_pred_base['fecha'].dt.weekday
df_pred_base['semana'] = df_pred_base['fecha'].dt.isocalendar().week


### Haciendo predicción con modelo creados

In [ ]:
df_forecasts = pd.DataFrame()

for sku in products_list:

    model = modelos[sku]
    df_sku_future = df_pred_base[df_pred_base['sku'] == sku].copy()

    df_sku_features = pd.get_dummies(df_sku_future[features], drop_first=True)
    df_sku_features = df_sku_features.reindex(columns=X_train.columns, fill_value=0)

    y_future_pred = model.predict(df_sku_features)
    df_sku_future['pred'] = y_future_pred
    
    df_forecasts = pd.concat([df_forecasts, df_sku_future], ignore_index=True)


In [12]:
importances = model.feature_importances_
feature_names = X_train.columns
importancia_df = pd.DataFrame({'feature': feature_names, 'importancia': importances})
importancia_df = importancia_df.sort_values(by='importancia', ascending=False)
print(importancia_df)

             feature  importancia
6         dia_semana     0.233018
13  tienda_Tienda_05     0.152363
15  tienda_Tienda_07     0.090191
2   indice_economico     0.065584
3                año     0.061910
7             semana     0.049817
4                mes     0.046110
16  tienda_Tienda_08     0.045698
5                dia     0.038651
1        temperatura     0.038159
0    precio_unitario     0.036780
11  tienda_Tienda_03     0.035675
12  tienda_Tienda_04     0.033116
8       region_Norte     0.023114
9         region_Sur     0.018624
14  tienda_Tienda_06     0.015744
10  tienda_Tienda_02     0.015446


In [13]:
importancias_modelos = {}
for sku in products_list:
    model = modelos[sku]
    feature_names = model.get_booster().feature_names
    importances = model.feature_importances_
    df_importancia = pd.DataFrame({'feature': feature_names, 'importance': importances})
    importancias_modelos[sku] = df_importancia.sort_values(by='importance', ascending=False)

In [14]:
# Tabla de pesos por producto
importancias_modelos['SKU_01']

,feature,importance
6,dia_semana,0.220392
4,mes,0.178401
9,region_Sur,0.119069
7,semana,0.117631
13,tienda_Tienda_05,0.115755
8,region_Norte,0.099601
16,tienda_Tienda_08,0.065884
14,tienda_Tienda_06,0.021309
10,tienda_Tienda_02,0.012300
1,temperatura,0.009610


### Grafico Resltados

In [15]:
def plot_forecast_por_tienda(df_future, tienda_objetivo):
    # Filtrar por tienda
    df_tienda = df_future[df_future['tienda'] == tienda_objetivo].copy()

    if df_tienda.empty:
        print(f"No hay datos para la tienda: {tienda_objetivo}")
        return

    # Graficar
    fig = px.line(
        df_tienda,
        x='fecha',
        y='pred',
        color='sku',
        markers=True,
        title=f'Proyección de ventas para la tienda {tienda_objetivo}',
        labels={
            'fecha': 'Fecha',
            'pred': 'Unidades Pronosticadas',
            'sku': 'Producto (SKU)'
        }
    )
    fig.update_layout(xaxis_title='Fecha', yaxis_title='Unidades Predichas')
    fig.show()

In [21]:
plot_forecast_por_tienda(df_forecasts, tienda_objetivo='Tienda_01')

### Guardar resultado

In [20]:
df_forecasts['pred']=df_forecasts['pred'].astype(int)
df_forecasts[["fecha","region","tienda",'sku','pred']].to_csv('data/forecast.csv')